In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['/home/vinnylg/projects/covid19datascience/notebooks', '/home/vinnylg/anaconda3/lib/python38.zip', '/home/vinnylg/anaconda3/lib/python3.8', '/home/vinnylg/anaconda3/lib/python3.8/lib-dynload', '', '/home/vinnylg/.local/lib/python3.8/site-packages', '/home/vinnylg/anaconda3/lib/python3.8/site-packages', '/home/vinnylg/.local/lib/python3.8/site-packages/IPython/extensions', '/home/vinnylg/.ipython', '/home/vinnylg/projects/covid19datascience']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input

default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output


In [4]:
read_cc = True and False
read_tb = True and False

In [5]:
cc = CasosConfirmados()
if read_cc:
    cc.read_excel()
    cc.save()
else:
    cc.load()

database: /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados.pkl
loading Casos Confirmados from pkl time elepsed:00:00:01


In [6]:
casosc = cc.get_casos()
casosc.loc[casosc['obito']!='SIM','obito'] = 'NAO'
print(f"casosc len: {len(casosc)}\n")
casosc.groupby('obito')[['sexo']].count()

casosc len: 1046917



,sexo
obito,
NAO,1021714
SIM,25203


In [7]:
tb_pacientes = TbPacientes()
if read_tb:
    tb_pacientes.update()

casost = tb_pacientes.get()
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
casost.groupby('obito')[['sexo']].count()

casost len: 1046917



,sexo
obito,
NAO,1021714
SIM,25203


In [8]:
casost.loc[~(
    casost['hash'].isin(casosc['hash']) |
    casost['hash'].isin(casosc['hash_less']) |
    casost['hash'].isin(casosc['hash_more']) |
    casost['hash_atend'].isin(casosc['hash_atend']) |
    casost['hash_less_atend'].isin(casosc['hash_atend']) |
    casost['hash_more_atend'].isin(casosc['hash_atend']) |
    casost['hash_diag'].isin(casosc['hash_diag'])
)]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [9]:
del casosc['identificacao']

In [10]:
tb_cols = ['identificacao','ibge_atend_pr','dt_com_obito','status','dt_com_recuperado','hash_less_atend','hash_more_atend']

casosct = pd.merge(casosc.loc[(casosc['obito']=="SIM")&(casosc['hash'].isin(casost['hash']))],casost[['hash']+tb_cols], how='left', on='hash')
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

25200
1021717


In [11]:
casosct =  casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash'])],casost[['hash']+ tb_cols], how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1045643
1892


In [12]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_less'])],casost[['hash_less']+ tb_cols].rename(columns={'hash_less':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1047537
0


In [13]:
casosct = casosct.append(pd.merge(casosc.loc[casosc['hash'].isin(casost['hash_more'])],casost[['hash_more']+ tb_cols].rename(columns={'hash_more':'hash'}), how='left', on='hash'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))

1047537
0


In [14]:
casosct = casosct.append(pd.merge(casosc,casost[['hash_diag']+ tb_cols], how='left', on='hash_diag'), ignore_index=True)
casosc = casosc.drop(index=casosc.loc[casosc['ordem'].isin(casosct['ordem'])].index)
print(len(casosct))
print(len(casosc))
casosct.loc[casosct['identificacao'].isna()]

1047537
0


,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [15]:
casosct['identificacao'] = casosct['identificacao'].astype(int)
casost.loc[~casost['identificacao'].isin(casosct['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito


In [16]:
id_duplicado = casosct.loc[casosct.duplicated('identificacao',keep=False)].sort_values('hash_diag')
ordem_duplicado = casosct.loc[casosct.duplicated('ordem',keep=False)].sort_values('hash_diag')
print(len(id_duplicado),len(ordem_duplicado))
all(id_duplicado['hash_diag'].values==ordem_duplicado['hash_diag'].values)

745 745


True

In [17]:
# casosct.loc[casosct.duplicated('identificacao',keep='last')]
casosct.loc[casosct['comunicacao']==pd.to_datetime('2020-03-11'),'comunicacao'] = pd.to_datetime('2020-03-12')
id_duplicado.loc[~id_duplicado['hash_diag'].isin(ordem_duplicado['hash_diag'])]

,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y


In [18]:
print(len(casosct))
casosct = casosct.drop_duplicates('identificacao',keep='last')
print('casos:',len(casosct))
print('obitos:',len(casosct.loc[casosct['obito']=='SIM']))

1047537
casos: 1046917
obitos: 25203


In [19]:
try:
    casosct = casosct_bkp.copy()
except:
    casosct_bkp = casosct.copy()
    
# casosct = casosct.sort_values(['comunicacao','nome','idade']).copy()
casosct = casosct.sort_values(['ordem']).copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito'].notna(),'data_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito'].notna(),'dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_obito']

casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[casosct['status'].notna(),'dt_com_recuperado'] 

# municipios = Municipios()
# casosct = pd.merge(casosct,municipios[['ibge7','regional','macro','uf','municipio']].rename(columns={'municipio':'municipio_pr'}),how='left',on='ibge7')

# casosct.loc[casosct['rs']==99, 'macro'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'regional'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'uf'] = 'Fora'
# casosct.loc[casosct['rs']==99, 'municipio_pr'] = 'Fora'

# macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
# macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
# casosct['macro'] = casosct['macro'].apply(lambda x: macros_labels_idx[x])

casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,728766
2,25203
3,292948


In [20]:
casosc = cc.get_casos()
casosct.loc[~casosct['ordem'].isin(casosc['ordem'])]

,ordem,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend_x,hash_more_atend_x,hash_diag,hash_obito,identificacao,ibge_atend_pr,dt_com_obito,status,dt_com_recuperado,hash_less_atend_y,hash_more_atend_y,evolucao,data_evolucao,data_com_evolucao


In [21]:
municipios = Municipios()
# municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
# municipios


# casosct['uf'] = ''
# casosct = casosct[['ordem','identificacao','uf','ibge7','ibge_atend_pr','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','excluir']]

In [22]:
# casos_fora = casosct.loc[casosct['ibge7']==9999999].copy()
# casos_fora['uf'] = casos_fora['mun_resid'].apply(lambda x: x.split('/')[-1])
# casos_fora['mun_resid'] = casos_fora['mun_resid'].apply(lambda x: x.split('/')[0])

# casos_fora = pd.merge(casos_fora.drop(columns=['ibge7']),municipios[['ibge7','mun_resid','uf']], on=['uf','mun_resid'], how='left')
# casos_fora.loc[casos_fora['ibge7'].isna(),'excluir'] = 'SIM'
# casos_fora.loc[casos_fora['ibge7'].isna(),'ibge7'] = 9999999

In [23]:
# casosct = casosct.drop(index=casos_fora.index)
casosct['uf_resid'] = "PR"
casosct.loc[casosct['ibge7']==9999999,'uf_resid'] = 'Fora'
# casosct = pd.concat([casosct,casos_fora])
# casosct['ibge7'] = casosct['ibge7'].apply(int)
# casosct= casosct.sort_values('ordem')
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,728766
2,25203
3,292948


In [24]:
#####
## novos casos confirmados
#

casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge7'].apply(lambda x: str(x)[:6])
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(lambda x: str(x)[:6])

casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna(), 'hash_obito'] = casosct.loc[(casosct['evolucao']==2) & casosct['data_com_evolucao'].notna()].apply(
        lambda row: normalize_hash(row['nome']) + date_hash(row['data_com_evolucao']), axis=1
)

In [25]:
casosct['hash'] = (casosct['nome'].apply(normalize_hash) +
                  casosct['idade'].astype(str) +
                  casosct['ibge_resid'] )

casosct['hash_less'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x - 1)) +
                        casosct['ibge_resid'] )

casosct['hash_more'] = ( casosct['nome'].apply(normalize_hash) +
                        casosct['idade'].apply(lambda x: str(x + 1)) +
                        casosct['ibge_resid'] )

casosct['hash_diag'] = (casosct['nome'].apply(normalize_hash) + casosct['dt_diag'].apply(date_hash))

casosct['ibge_resid'] = casosct['ibge_resid'].apply(int)
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)

casosct = casosct[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_diag']]#,'hash_atend','hash_less_atend','hash_more_atend','hash_diag','hash_obito']]
casosct = casosct.set_index('identificacao')
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,728766
2,25203
3,292948


In [26]:
casosct.head()

,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_diag
identificacao,,,,,,,,,,,,,,,,,,
8838,-1,PR,410550,4105508,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,NaT,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
10932,-1,PR,410690,4106902,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
10933,-1,PR,410690,4106902,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
19775,-1,PR,410690,4106902,LUIZ ANTONIO CAXAMBU FREITAS,M,54,GENOPRIMER,2020-03-11,2020-03-12,2020-03-09,1,NaT,NaT,LUIZANTONIOCAXAMBUFREITAS54410690,LUIZANTONIOCAXAMBUFREITAS53410690,LUIZANTONIOCAXAMBUFREITAS55410690,LUIZANTONIOCAXAMBUFREITAS11032020
22343,-1,PR,410690,4106902,MARIANA CAXAMBU FREITAS,F,25,GENOPRIMER,2020-03-11,2020-03-12,2020-03-01,1,NaT,NaT,MARIANACAXAMBUFREITAS25410690,MARIANACAXAMBUFREITAS24410690,MARIANACAXAMBUFREITAS26410690,MARIANACAXAMBUFREITAS11032020


In [27]:
novos_casos = pd.read_excel(join(default_input,'novos_casos.xlsx')).rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'nome_exame': 'laboratorio',
    'data_liberacao':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao'
})
novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

casosct.loc[casosct['uf_resid']!='PR','uf_resid'] = 'Fora'

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos.index = list(range(casosct.iloc[-1].name + 1,casosct.iloc[-1].name + 1 + len(novos_casos)))
casosct = casosct.append(novos_casos)
print(len(casosct))
casosct.groupby('evolucao')[['sexo']].count()

1054127


,sexo
evolucao,
1,729518
2,25251
3,299358


In [28]:
novos_obitos = pd.read_excel(join(default_input,'novos_obitos.xlsx')).rename(columns={
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = date.today()

novos_obitos = novos_obitos[['data_evolucao','data_com_evolucao','hash']]
len(novos_obitos)

172

In [29]:
casosct_obitos = casosct.loc[casosct['hash'].isin(novos_obitos['hash'])]
casosct_obitos = casosct_obitos.drop(columns=['data_evolucao','data_com_evolucao'])
casosct_obitos_index = casosct_obitos.index
casosct_obitos = pd.merge(casosct_obitos,novos_obitos[['hash','data_evolucao','data_com_evolucao']], on='hash', how='left')
casosct_obitos['evolucao'] = 2
casosct_obitos.index = casosct_obitos_index
len(casosct_obitos)

172

In [30]:
casosct = casosct.drop(index=casosct_obitos_index).append(casosct_obitos)
print(len(casosct))
# casosct.to_pickle('casosct.pkl')
casosct.groupby('evolucao')[['sexo']].count()

1054127


,sexo
evolucao,
1,729502
2,25375
3,299250


In [31]:
macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casosct = pd.merge(casosct.rename(columns={'ibge_resid':'ibge'}),municipios,on='ibge',how='left').rename(columns={'ibge':'ibge_resid'})
casosct['municipio_pr'] = casosct['municipio']
casosct.loc[casosct['ibge_resid']==999999,'municipio_pr'] = 'Fora'

In [32]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casosct.groupby(['evolucao','uf_resid'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.048.377,5.750,1.054.127
OBITOS,25.231,144,25.375
RECUPERADOS,727.342,2.160,729.502
ATIVOS,295.804,3.446,299.250


In [33]:
total_macros = casosct.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,487.593,12.866,327.153,147.574
Oeste,216.235,4.195,182.786,29.254
Noroeste,168.651,3.602,124.871,40.178
Norte,175.898,4.568,92.532,78.798
Fora,5.750,144,2.160,3.446


In [34]:
total_regionais = casosct.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,33.124,797,17.495,14.832
2,Metropolitana,314.374,9.085,222.064,83.225
3,Ponta Grossa,61.041,1.448,40.406,19.187
4,Irati,14.138,236,10.137,3.765
5,Guarapuava,32.388,651,18.817,12.920
6,União da Vitória,10.383,203,7.659,2.521
7,Pato Branco,28.027,624,23.523,3.880
8,Francisco Beltrão,37.878,634,30.224,7.020
9,Foz do Iguaçu,54.104,1.099,42.061,10.944


In [35]:
municipios
total_municipios = casosct.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

# #comentar 3 linhas abaixo para remover totais
# total_municipios = total_municipios.append(pd.Series(total_municipios.sum(0),name=('','TOTAL','GERAL','','')))
# pr = (total_municipios.loc[('','TOTAL','GERAL','','')].values - total_municipios.loc['Fora'].values)[0]
# total_municipios = total_municipios.append(pd.Series(pr, index=total_municipios.columns, name=(('','TOTAL','PR','',''))))

# total_municipios = total_municipios.reset_index()
# total_municipios.to_excel('total_municipios.xlsx')
total_municipios

evolucao,macro,rs,regional,ibge7,municipio_pr,CASOS,OBITOS,RECUPERADOS,ATIVOS
0,Leste,1,Paranaguá,4101200,Antonina,2.199,56,1.987,156
1,Leste,1,Paranaguá,4109500,Guaraqueçaba,592,14,477,101
2,Leste,1,Paranaguá,4109609,Guaratuba,4.205,146,3.731,328
3,Leste,1,Paranaguá,4115705,Matinhos,2.622,85,2.272,265
4,Leste,1,Paranaguá,4116208,Morretes,1.845,49,1.468,328
...,...,...,...,...,...,...,...,...,...
395,Norte,22,Ivaiporã,4122172,Rio Branco do Ivaí,257,8,224,25
396,Norte,22,Ivaiporã,4122651,Rosário do Ivaí,375,2,336,37
397,Norte,22,Ivaiporã,4123857,Santa Maria do Oeste,570,22,470,78
398,Norte,22,Ivaiporã,4125001,São João do Ivaí,782,16,674,92


In [36]:
casosct['regional'] = casosct['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casosct['regional']
casosct.columns

Index(['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo',
       'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao',
       'data_evolucao', 'data_com_evolucao', 'hash', 'hash_less', 'hash_more',
       'hash_diag', 'ibge7', 'rs', 'regional', 'macro', 'municipio', 'uf',
       'populacao', 'estado', 'municipio_pr'],
      dtype='object')

In [37]:
casos_confirmados = casosct[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

/home/vinnylg/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [38]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [39]:
writer = pd.ExcelWriter("casos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in casos_confirmados['macro'].unique():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==macro].copy()
    casos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,casos_macro,macros_idx_labels[macro])


writer.save()

In [40]:
writer = pd.ExcelWriter("obitos_confirmados.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for macro in obitos_confirmados['macro'].unique():
    obitos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==macro].copy()
    obitos_macro.to_excel(writer,macros_idx_labels[macro])
    fit_cols(writer,obitos_macro,macros_idx_labels[macro])


writer.save()

In [41]:
writer = pd.ExcelWriter("casos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in casos_confirmados['regional'].unique():
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [42]:
writer = pd.ExcelWriter("obitos_confirmados_regionais.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in obitos_confirmados['regional'].unique():
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()